# Data Gathering

The 2 subreddits chosen are SucideWatch and DomesticViolence.

SucideWatch post are mainly people talking about their sucides experiences. 

DomesticViolence post are mainly people talking about their experiences of domestic violence.

In [52]:
import numpy as np
import pandas as pd
import requests
import time
import datetime as dt
import json

In [53]:
sub = 'domesticviolence'
size = '500'

In [54]:
url = f'https://api.pushshift.io/reddit/search/submission?subreddit={sub}&size={size}'

In [55]:
r = requests.get(url)

In [56]:
r.status_code

200

In [57]:
data = r.json()

In [58]:
type(data['data'])

list

In [59]:
len(data['data'])

500

In [60]:
data['data'][0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': '[deleted]',
 'author_flair_background_color': '',
 'author_flair_css_class': None,
 'author_flair_text': None,
 'author_flair_text_color': 'dark',
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1592842102,
 'domain': 'self.domesticviolence',
 'edited': 1592842631,
 'full_link': 'https://www.reddit.com/r/domesticviolence/comments/hdv5pl/my_abusive_ex_showed_up_at_my_work_but_i_think_it/',
 'gildings': {},
 'id': 'hdv5pl',
 'is_crosspostable': False,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': False,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_css_class': 'new',
 'link_flair_richtext': [],
 'link_flair_text': '[new]',
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 0,
 'num_crossposts': 0,
 'over_

In [61]:
# References used to create this code:
# https://github.com/pushshift/api
# Brian Collins, General Assembly instructor
def pushshift(subreddit, post_type='submission', loops=1, skip=30):
# subreddit: str, name of subreddit to search for
# post_type: {'submission', 'comment'}, type of post to search for
# loops: int, number of times to request posts
# size: int, number of posts per request (max 500 per pushshift api guide)
# skip: int, number of days back to search in each loop
        # increase if too many duplicate posts are returned, decrease if you want to skip fewer posts
    # data fields to return for submissions
    subfields = ['author', 'author_fullname', 'created_utc', 'id', 'num_comments', 'permalink',
                 'score', 'selftext', 'subreddit', 'title', 'url', 'is_self']
    # data fields to return for comments
    comfields = ['author', 'author_fullname', 'body', 'created_utc', 'id', 'parent_id',
                'permalink', 'score', 'subreddit']
    # instantiate list for posts data
    list_posts = []
    url_stem = "https://api.pushshift.io/reddit/search/{}/?subreddit={}".format(post_type, subreddit)
    # skip a minimum of 1 day
    after = 1
    # check before requesting data
    if post_type not in ['submission', 'comment']:
        print("post_type must be 'submission' or 'comment'")
        return None
    for i in range(loops):
        # add parameters to url to skip posts (after could be used to match up to post at end of previous loop if skip = 0)
        url = '{}&after={}d'.format(url_stem, skip * i + after)
        # monitor status as loops run
        print(i, url)
        # get data
        res = requests.get(url)
        # add dictionaries for posts to list_posts
        list_posts.extend(res.json()['data'])
        # be polite
        time.sleep(1)
    # turn list_posts (a list of dictionaries where each dictionary contains data on one post) into a dataframe
    df_posts = pd.DataFrame.from_dict(list_posts)
    # filter fields for submissions or comments
    if post_type == 'submission':
        df_posts = df_posts[subfields]
    elif post_type == 'comment':
        df_posts = df_posts[comfields]
#     else:
#         print("post_type must be 'submission' or 'comment'")
#         return None
    # drop any duplicates
    #df_posts.drop_duplicates(inplace=True)
    # add a field identifying submissions or comments
    #df_posts['post_type'] = post_type
    return df_posts

In [62]:
domesticviolence_subs = pushshift('domesticviolence', loops=40, skip=1)
print('shape', domesticviolence_subs.shape)

0 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=1d
1 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=2d
2 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=3d
3 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=4d
4 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=5d
5 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=6d
6 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=7d
7 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=8d
8 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=9d
9 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=10d
10 https://api.pushshift.io/reddit/search/submission/?subreddit=domesticviolence&after=11d
11 https://api.pus

In [63]:
domesticviolence_subs.to_csv('data/domesticviolence_subs-pushshift.csv')

In [64]:
suicidewatch_subs = pushshift('suicidewatch', loops=40, skip=1)
print('shape', suicidewatch_subs.shape)

0 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=1d
1 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=2d
2 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=3d
3 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=4d
4 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=5d
5 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=6d
6 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=7d
7 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=8d
8 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=9d
9 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=10d
10 https://api.pushshift.io/reddit/search/submission/?subreddit=suicidewatch&after=11d
11 https://api.pushshift.io/reddit/search/submission/?subreddi

In [66]:
suicidewatch_subs.to_csv('data/suicidewatch_subs-pushshift.csv')